# Background
The Delta Lake [`replaceWhere`](https://mungingdata.com/delta-lake/updating-partitions-with-replacewhere/) option allows users to selectively apply updates to specific data partitions rather than to full lakes, which may result in significant speed gains. This notebook briefly illustrates the usage of `replaceWhere` option. For more details, see:
- [Selectively updating Delta partitions with replaceWhere](https://mungingdata.com/delta-lake/updating-partitions-with-replacewhere/) (this notebook will be following the example from this blog)
- [Selectively overwrite data with Delta Lake](https://docs.databricks.com/delta/selective-overwrite.html)
- [Table batch reads and writes: overwrite](https://docs.delta.io/latest/delta-batch.html#overwrite)

# Example

## 1. Load some Data

In [2]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
import pyspark
from delta import *

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

df = spark.read.options(header='True', charset='UTF8')\
    .csv("../../data/people_countries.csv")\
    .withColumn("continent", lit(None).cast(StringType()))

df.show()

+----------+---------+---------+---------+
|first_name|last_name|  country|continent|
+----------+---------+---------+---------+
|   Ernesto|  Guevara|Argentina|     null|
|  Vladimir|    Putin|   Russia|     null|
|     Maria|Sharapova|   Russia|     null|
|     Bruce|      Lee|    China|     null|
|      Jack|       Ma|    China|     null|
+----------+---------+---------+---------+



## Partition on Country
Now we'll repartition the DataFrame on `country` and write it to disk in the Delta Lake format, partitioned by `country`.

In [3]:
from pyspark.sql.functions import col
deltaPath = "../../data/people_countries_delta/"

(df.repartition(col("country")) 
  .write 
  .partitionBy("country") 
  .format("delta") 
  .mode("overwrite") 
  .save(deltaPath)
  )


23/06/23 13:28:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Now we write a function to add `continent` values to a DataFrame based on the value of `country`.

In [4]:
from pyspark.sql.functions import when, col

def withContinent(df):
    return df.withColumn(
        "continent",
        when(col("country") == "Russia", "Europe")
          .when(col("country") == "China", "Asia")
          .when(col("country") == "Argentina", "South America")
      )

Here's where `replaceWhere` comes in. Suppose we only want to populate the `continent` column when `country == 'China'`.

In [5]:
df = spark.read.format("delta").load(deltaPath)
df = df.where(col("country") == "China").transform(withContinent)

(df.write
  .format("delta")
  .option("replaceWhere", "country = 'China'")
  .mode("overwrite")
  .save(deltaPath))

In [6]:
spark.read.format("delta").load(deltaPath).show(truncate=False)

+----------+---------+---------+---------+
|first_name|last_name|country  |continent|
+----------+---------+---------+---------+
|Bruce     |Lee      |China    |Asia     |
|Jack      |Ma       |China    |Asia     |
|Ernesto   |Guevara  |Argentina|null     |
|Vladimir  |Putin    |Russia   |null     |
|Maria     |Sharapova|Russia   |null     |
+----------+---------+---------+---------+



Let's see what happened.

In [8]:
import json
with open("../../data/people_countries_delta/_delta_log/00000000000000000001.json", "r") as f:
    for line in f:
        data = json.loads(line)
        if "add" in data or "remove" in data:
            print(json.dumps(data, indent=4))

{
    "add": {
        "path": "country=China/part-00000-2f823649-f7af-45e7-95cc-fce354972434.c000.snappy.parquet",
        "partitionValues": {
            "country": "China"
        },
        "size": 1002,
        "modificationTime": 1687544927388,
        "dataChange": true,
        "stats": "{\"numRecords\":2,\"minValues\":{\"first_name\":\"Bruce\",\"last_name\":\"Lee\",\"continent\":\"Asia\"},\"maxValues\":{\"first_name\":\"Jack\",\"last_name\":\"Ma\",\"continent\":\"Asia\"},\"nullCount\":{\"first_name\":0,\"last_name\":0,\"continent\":0}}"
    }
}
{
    "remove": {
        "path": "country=China/part-00000-5b20d31c-1a49-47f0-a5b1-0f2e5b422753.c000.snappy.parquet",
        "deletionTimestamp": 1687544926730,
        "dataChange": true,
        "extendedFileMetadata": true,
        "partitionValues": {
            "country": "China"
        },
        "size": 929
    }
}


We can see that only the `country=China/part-00000-87aebbc2-aff3-4bd6-b369-aa9aacbb93be.c000.snappy.parquet` file was modified. The other partitions were not.

For more details, read the [blog post](https://mungingdata.com/delta-lake/updating-partitions-with-replacewhere/).